# Problema a ser resolvido:

------

- Tenho uma carteira com 10 ações. Dado a distribuição dos retornos dessa carteira no último ano:

    * Qual o máximo que eu posso perder no próximo ano, com uma confiança de 95%?
    * Qual a probabilidade de eu obter lucro?

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
import yfinance as yf
from numpy import linalg as LA

In [ ]:
#pegando dados

lista_acoes = ['WEGE3', 'PCAR3', 'LREN3', 'PETR4', 'VALE3']
lista_acoes = [acao + ".SA" for acao in lista_acoes]

data_final = dt.datetime.now()
data_inicial = data_final - dt.timedelta(days=300)

precos = yf.download(lista_acoes, data_inicial, data_final)['Adj Close']

In [ ]:
#calculando retornos pegando matriz de covariância 

retornos = precos.pct_change().dropna()
media_retornos = retornos.mean()
matriz_covariancia = retornos.cov()
pesos_carteira = np.full(len(lista_acoes), 1/len(lista_acoes))
numero_acoes = len(lista_acoes)

pesos_carteira

# Fórmula retornos sintéticos
---
Retornos_sintéticos = média_retornos + Rpdf × L.

* média_retornos = Média dos retornos.
* Rpdf = Matriz aleatória gerada por alguma função de densidade de probabilidade.
* L = Matriz triangular inferior proveniente de uma decomposição de Cholesky, usando como base a matriz de covariância dos dados originais.


# Por que fazer isso?
---

Nós assumimos que a distribuição de retornos é uma distribuição normal multivariada (isso é uma expansão da distribuição normal).

Quando geremos retornos aleátorios de cada ativo (Weg, Vale, etc), nós criamos vetores aleatórios descorrelacionados entre si. Para corrigir isso, precisamos correlacionar esses retornos (no mundo real isso é o que acontece) e, para isso, usamos a matriz triângular obtida a partir da covariância.

In [ ]:
#premissas montecarlo

numero_simulacoes = 10000
dias_projetados = 252 
capital_inicial = 1000

In [ ]:
#gerando retorno médio
retorno_medio = retornos.mean(axis = 0).to_numpy() 
matriz_retorno_medio = retorno_medio * np.ones(shape = (dias_projetados, numero_acoes))

In [ ]:
#gerando L
L = LA.cholesky(matriz_covariancia)
L

In [ ]:
#gerando simulações

retornos_carteira = np.zeros([dias_projetados, numero_simulacoes]) #cada coluna é uma simulação
montante_final = np.zeros(numero_simulacoes)

for s in range(numero_simulacoes):

    Rpdf = np.random.normal(size=(dias_projetados, numero_acoes)) 
    
    retornos_sintéticos = matriz_retorno_medio + np.inner(Rpdf, L) #unica coisa random é o Rpdf
    
    retornos_carteira[:, s] = np.cumprod(np.inner(pesos_carteira, 
                                                  retornos_sintéticos) + 1) * capital_inicial
    montante_final[s] = retornos_carteira[-1, s]
    
    
retornos_carteira

In [ ]:
plt.plot(retornos_carteira, linewidth=1)
plt.ylabel('Dinheiro')
plt.xlabel('Dias')
plt.show()

In [ ]:
montante_99 = str(np.percentile(montante_final, 1))
montante_95 = str(np.percentile(montante_final, 5))
montante_mediano = str(np.percentile(montante_final, 50))
cenarios_com_lucro = str((len(montante_final[montante_final > 1000])/
                                len(montante_final)) * 100) + "%"

In [ ]:
print(f'''Ao investir R$ 1000,00 na carteira {lista_acoes}, 
podemos esperar esses resultados para os próximo ano, 
utilizando o método de Monte Carlo com 10 mil simulações:

Com 50% de probabilidade, o montante será maior que R$ {montante_mediano}. 

Com 95% de probabilidade, o montante será maior que R$ {montante_95}.

Com 99% de probabilidade, o montante será maior que R$ {montante_99}.

Em {cenarios_com_lucro} dos cenários, foi possível obter lucro no próximo ano.''')

In [ ]:
-50%

# Utilidades

A partir dessas métricas, é possível calcular coisas como o VAR da carteira para diferentes intervalos de confiança e medir se o risco que você está correndo é compatível com o que você aguenta perder. No caso dessa carteira, uma queda de 27,5% está dentro dos 95% dos cenários mais possíveis de acontecer no próximo ano.

In [ ]:
config = dict(histtype = "stepfilled", alpha = 0.8, density = False, bins = 150)
fig, ax = plt.subplots()
ax.hist(montante_final, **config)
ax.xaxis.set_major_formatter('R${x:.0f}')
plt.title('Distribuição montantes finais com simulação MC')
plt.xlabel('Montante final (R$)')
plt.ylabel("Frequência")
plt.show()